# 🥇 Phase 3: The Gold Layer (Customer 360 Feature Store)
**Project:** "Olist-Next" Hyper-Personalized Retention Engine
**Layer:** Gold (Aggregated/ML Ready)

## 🧠 Objective
We construct the final training dataset for the Churn Prediction Model.
We aggregate transaction history into a single **Customer Profile** (Customer 360) with the following features:

* **Recency:** Days since the last purchase (Relative to the dataset's max date).
* **Frequency:** Total number of completed orders.
* **Monetary:** Total Lifetime Value (LTV) calculated from payments.
* **Behavioral:** Their favorite product category and average review score.
* **Target (Label):** `is_churn_risk` (1 if Recency > 90 days, else 0).

**Note on Time:** Since this is a historical dataset (ending in 2018), we do NOT use the current date. We calculate a dynamic "Anchor Date" (the last recorded transaction in the system) to calculate Recency.

### Setup & Load (Python)
Load all necessary Silver tables and imports.

In [0]:
from pyspark.sql.functions import col, max, countDistinct, sum, avg, datediff, lit, row_number, count, desc, when
from pyspark.sql.window import Window

# Set Context
spark.sql("USE CATALOG olist_hackathon")

# Load Silver Tables
df_orders = spark.table("silver.orders")
df_items = spark.table("silver.order_items")
df_payments = spark.table("silver.payments")
df_reviews = spark.table("silver.reviews")
df_products = spark.table("silver.products")
df_customers = spark.table("silver.customers")

print("✅ Silver tables loaded.")

### Determine "Anchor Date" (Python)
We dynamically find the "present" moment in the dataset to calculate Recency correctly.

In [0]:
# 1. Calculate the latest date in the entire dataset
max_date_row = df_orders.select(max("order_purchase_timestamp")).collect()
anchor_date = max_date_row[0][0]

print(f"⚓ Anchor Date established: {anchor_date}")
print("   (All 'Recency' calculations will be relative to this date)")

### Feature Logic - RFM & Satisfaction (Python)
This step calculates Recency, Frequency, Monetary, and Review Scores.

In [0]:
def calculate_rfm_features():
    print("⏳ Calculating Base RFM Metrics...")

    # Join strategy: Customers -> Orders -> Payments
    # We use 'customer_unique_id' as the primary key for the profile
    
    df_rfm = (
        df_customers
        .join(df_orders, "customer_id", "inner")
        .join(df_payments, "order_id", "inner")
        # --- Aggregation ---
        .groupBy("customer_unique_id")
        .agg(
            max("order_purchase_timestamp").alias("last_purchase_date"),
            countDistinct("order_id").alias("frequency"),
            sum("payment_value").alias("monetary_value")
        )
        # --- Feature Engineering ---
        .withColumn("recency_days", datediff(lit(anchor_date), col("last_purchase_date")))
    )
    
    # Calculate Average Review Score separately (to handle orders with multiple reviews)
    df_avg_review = (
        df_orders
        .join(df_reviews, "order_id", "inner")
        .join(df_customers, "customer_id", "inner")
        .groupBy("customer_unique_id")
        .agg(avg("review_score").alias("avg_review_score"))
    )
    
    # Combine RFM with Reviews
    return df_rfm.join(df_avg_review, "customer_unique_id", "left")

df_features_base = calculate_rfm_features()
print(f"✅ Base Features calculated. Count: {df_features_base.count()}")

### Feature Logic - Favorite Category (Python)
This is trickier. We need to find the "Mode" (most frequent) category for each user.

In [0]:
def calculate_favorite_category():
    print("⏳ Calculating Favorite Categories...")
    
    # 1. Big Join: Customer -> Order -> Item -> Product
    df_cust_cat = (
        df_customers
        .join(df_orders, "customer_id")
        .join(df_items, "order_id")
        .join(df_products, "product_id")
        .select("customer_unique_id", "category_name")
    )
    
    # 2. Group by Customer + Category to get counts
    df_cat_counts = (
        df_cust_cat
        .groupBy("customer_unique_id", "category_name")
        .agg(count("*").alias("cat_count"))
    )
    
    # 3. Window Function to rank categories by count for each user
    window_spec = Window.partitionBy("customer_unique_id").orderBy(col("cat_count").desc())
    
    df_top_category = (
        df_cat_counts
        .withColumn("rank", row_number().over(window_spec))
        .filter(col("rank") == 1) # Keep only the #1 category
        .select("customer_unique_id", col("category_name").alias("favorite_category"))
    )
    
    return df_top_category

df_features_category = calculate_favorite_category()
print("✅ Favorite Categories calculated.")

### Final Assembly & Target Creation (Python)
Combine everything and define the Churn Label.

In [0]:
# 1. Join Base Features with Favorite Category
df_customer_360 = (
    df_features_base
    .join(df_features_category, "customer_unique_id", "left")
    # Fill nulls for categories (some orders might not have items linked properly)
    .fillna({"favorite_category": "Unknown", "avg_review_score": 3.0}) 
)

# 2. Define Target Variable (Churn Risk)
# Logic: If they haven't bought in 90 days, we consider them "At Risk" (1)
df_final = df_customer_360.withColumn(
    "is_churn_risk", 
    when(col("recency_days") > 90, 1).otherwise(0)
)

print("✅ Final Customer 360 Dataset Assembled.")

### Write to Gold (Python)
Save the result as a Delta table.

In [0]:
target_table = "olist_hackathon.gold.customer_360"

print(f"⏳ Writing to {target_table}...")

(df_final.write
 .format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(target_table)
)

print(f"🎉 Success! Gold table created: {target_table}")

### Verification 
Display the profile to ensure the logic looks sound.

In [0]:
# Show top 10 High Value customers
display(
    spark.table("olist_hackathon.gold.customer_360")
    .orderBy(col("monetary_value").desc())
    .limit(10)
)